# NiN块

NiN块是网络中的基本构建块，它由一个卷积层加两个充当全连接层的1x1卷积层组成。

In [1]:
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import nn

def nin_block(num_channels, kernel_size, strides, padding):
    blk = nn.Sequential()
    blk.add(nn.Conv2D(num_channels, kernel_size, strides, padding, activation='relu'),
            nn.Conv2D(num_channels, kernel_size=1, activation='relu'),
            nn.Conv2D(2*num_channels, kernel_size=1, activation='relu'))
    return blk

NiN模型

In [5]:
net=nn.Sequential()
net.add(nin_block(96, kernel_size=11, strides=4, padding=0), 
        nn.MaxPool2D(pool_size=3, strides=2),
        nin_block(256, kernel_size=5, strides=1, padding=2), 
        nn.MaxPool2D(pool_size=3, strides=2),
        nin_block(384, kernel_size=3, strides=1, padding=1), 
        nn.MaxPool2D(pool_size=3, strides=2), 
        nn.Dropout(0.5),
        nin_block(10, kernel_size=3, strides=1, padding=1),
        nn.GlobalAvgPool2D(),  # 将四维输出转化为二维输出
        nn.Flatten())

In [6]:
X=nd.random.uniform(shape=(1,1,224,224))
net.initialize()
for layer in net:
    X=layer(X)
    print(layer.name,'output shape:\t',X.shape)

sequential7 output shape:	 (1, 192, 54, 54)
pool0 output shape:	 (1, 192, 26, 26)
sequential8 output shape:	 (1, 512, 26, 26)
pool1 output shape:	 (1, 512, 12, 12)
sequential9 output shape:	 (1, 768, 12, 12)
pool2 output shape:	 (1, 768, 5, 5)
dropout0 output shape:	 (1, 768, 5, 5)
sequential10 output shape:	 (1, 20, 5, 5)
pool3 output shape:	 (1, 20, 1, 1)
flatten0 output shape:	 (1, 20)


获取数据和训练模型

In [9]:
lr,num_epochs,batch_size=0.1,5,128
ctx=d2l.try_gpu()
net.initialize(init=init.Xavier(),ctx=ctx,force_reinit=True)
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size,resize=224)
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on cpu(0)


KeyboardInterrupt: 